In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import nltk, re

from nltk.corpus import stopwords

In [2]:
trainData = pd.read_csv('../data/raw/training.csv')
testData = pd.read_csv('../data/raw/test.csv')
valData = pd.read_csv('../data/raw/validation.csv')

print(trainData.shape)
print(testData.shape)
print(valData.shape)

(16000, 2)
(2000, 2)
(2000, 2)


In [9]:
trainData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16000 non-null  object
 1   label   16000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.1+ KB


In [6]:
stop_words = set(stopwords.words('english'))
def cleanText(text : str) -> str:
    text = text.lower()
    text = re.sub(r'^[a-zA-Z0-9]', '', text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

trainData["text"] = trainData["text"].apply(cleanText)
testData['text'] = testData['text'].apply(cleanText)
valData['text'] = valData['text'].apply(cleanText)